## Asteroid conjunction finder

Find close approaches between asteroid positions (obtained dynamically from MPC ephemeris service) and NGC DSO catalog (cached locally from Vizier table)

In [81]:
# Parameters, edit to taste

# Date & time for ephemeris lookup

OBS_TIME = '2025-01-13 20:00:00'

# coords of home location (used to calculate altitude of conjunction)

OBS_LAT = 52.9
OBS_LONG = 0.5

# mimimum angular size of DSO to be visible in image
DSO_MIN_SIZE = 2  # 5% of Seestar S50 FoV width

# maximum magnitude for DSO to be visible 
DSO_MAX_MAG = 15  # approx limiting magnitude for Seestar S50

# Max angular distance between asteroid and DSO
MAX_SEPARATION_ARCMIN = 40 # approx frame width in Seestar S50

In [63]:
# list of largest asteroids to search

ASTEROIDS = ['Ceres',
                      '4 Vesta',
                       '2 Pallas',
                       '10 Hygiea',
                       '704 Interamnia',
                       '52 Europa',
                       '511 Davida',
                       '87 Sylvia',
                       '15 Eunomia',
                       '31 Euphrosyne',
                       '65 Cybele',
                       '624 Hektor',
                       '3 Juno',
                       '451 Patientia',
                       '107 Camilla',
                       '324 Bamberga',
                       '16 Psyche',
                       '88 Thisbe',
                       '48 Doris',
                       '19 Fortuna',
                       '121 Hermione',
                       '24 Themis',
                       '94 Aurora',
                       '29 Amphitrite',
                       '13 Egeria',
                       '130 Elektra',
                       '7 Iris',
                       '6 Hebe',
                       '375 Ursula',
                       '702 Alauda',
                       '45 Eugenia',
                       '41 Daphne',
                       '423 Diotima',
                       '259 Aletheia',
                       '372 Palma',
                       '9 Metis',
                       '532 Herculina',
                       '354 Eleonora',
                       '128 Nemesis'
]

In [ ]:
# Fetch ngc catalog from CDS vizier server
# This code only needs to be run once to cache the table locally

# from astroquery.vizier import Vizier

# vizier = Vizier()
# vizier.ROW_LIMIT = -1

# ngc_tables = vizier.get_catalogs('VII/118/ngc2000')

# # save ngc table locally for future reference

# ngc_tables[0].write('./ngc_catalog.ecsv')


In [ ]:
# create SkyCoord catalog of ngc DSOs for searching

from astropy.io import ascii

# load locally cached ngc table 
ngc_table = ascii.read('./ngc_catalog.ecsv')

# filter table to size and magnitude limits

filtered_ngc_table = ngc_table[ngc_table['size'] > DSO_MIN_SIZE]
filtered_ngc_table = filtered_ngc_table[filtered_ngc_table['mag'] < DSO_MAX_MAG]

In [38]:
from astropy.coordinates import SkyCoord
import astropy.units as u

ngc_catalog = SkyCoord(ra = filtered_ngc_table['RAB2000'], dec = filtered_ngc_table['DEB2000'],
                       unit = (u.hourangle, u.deg))



In [ ]:
# Query MPC Horizons service for asteroid ephemeris data

from astroquery.mpc import MPC
from astropy.table import vstack

ephemerides = [MPC.get_ephemeris(a, start = OBS_TIME, number = 1) for a in ASTEROIDS]
ephemerides = vstack(ephemerides)

# create SkyCoord catalog for search
ephemerides_cat = SkyCoord(ra = ephemerides['RA'], dec = ephemerides['Dec'], unit = u.deg)

In [92]:
# Find sky conjunctions of asteroid positions and NGC catalog

from astropy.coordinates import AltAz, EarthLocation

idx_ngc, idx_asteroid, sep2d, _ = ephemerides_cat.search_around_sky(ngc_catalog, seplimit = MAX_SEPARATION_ARCMIN * u.arcmin)

home = EarthLocation(lat = OBS_LAT * u.deg, lon = OBS_LONG * u.deg)
home_altaz = AltAz(obstime = OBS_TIME, location = home)

print('Predicted conjunctions for ', OBS_TIME)
for ingc, ia, sep in zip(idx_ngc, idx_asteroid, sep2d):
    ngc = filtered_ngc_table[ingc]
    ngc_loc = ngc_catalog[ingc]
    ephem = ephemerides_cat[ia]
    altaz_ast =  ephem.transform_to(home_altaz)
    altaz_ngc = ngc_loc.transform_to(home_altaz)
    print(f"NGC {ngc['Name']}, {ngc['Type']}, (alt={altaz_ngc.alt.degree:.1f}) <> {ASTEROIDS[ia]} (alt={altaz_ast.alt.degree:.1f}), sep = {sep:.2f}")
    

Predicted conjunctions for  2025-01-13 20:00:00
NGC 428, Gx, (alt=30.4) <> 107 Camilla (alt=30.2), sep = 0.49 deg
NGC 514, Gx, (alt=42.4) <> 10 Hygiea (alt=42.5), sep = 0.37 deg
NGC 936, Gx, (alt=34.3) <> 511 Davida (alt=34.8), sep = 0.64 deg
NGC 941, Gx, (alt=34.3) <> 511 Davida (alt=34.8), sep = 0.53 deg
NGC 955, Gx, (alt=34.4) <> 511 Davida (alt=34.8), sep = 0.60 deg
NGC I 443, Nb, (alt=47.0) <> 88 Thisbe (alt=47.3), sep = 0.35 deg
NGC 5705, Gx, (alt=-36.7) <> 451 Patientia (alt=-37.1), sep = 0.43 deg
